## **Camera to Machine Calibration** ##

In [1]:
import sys
import numpy as np
import time
import math
import json
import ipywidgets as widgets
import cv2
import os

In [2]:
#print(os.getcwd()) 
print(sys.path)
#sys.path.append('path_to_your_science_jubilee_directory')

['c:\\Users\\titan\\anaconda3\\envs\\axo\\python313.zip', 'c:\\Users\\titan\\anaconda3\\envs\\axo\\DLLs', 'c:\\Users\\titan\\anaconda3\\envs\\axo\\Lib', 'c:\\Users\\titan\\anaconda3\\envs\\axo', '', 'c:\\Users\\titan\\anaconda3\\envs\\axo\\Lib\\site-packages', 'C:\\science-jubilee\\src', 'c:\\Users\\titan\\anaconda3\\envs\\axo\\Lib\\site-packages\\win32', 'c:\\Users\\titan\\anaconda3\\envs\\axo\\Lib\\site-packages\\win32\\lib', 'c:\\Users\\titan\\anaconda3\\envs\\axo\\Lib\\site-packages\\Pythonwin']


In [1]:
#from science_jubilee.tools.Camera import Camera
from science_jubilee.Machine import Machine
from science_jubilee.tools.Tool import Tool
from science_jubilee.decks.Deck import Deck
from science_jubilee.labware.Labware import Labware, Location
from science_jubilee.tools.Pipette import Pipette
from science_jubilee.bo.newcamera import Cam

In [4]:
deck = Deck("lab_automation_deck") # We are not using the deck, so operate any deck.json 
tiprack = deck.load_labware(labware_filename = 'opentrons_96_tiprack_20ul.json', slot = 1)   # Load an opentrons 20uL
tiprack.manual_offset([[269, 221.6],[170, 221.6],[170, 284.6]], save = True)   # Save through list of list. List of coordinates should be A1 A12 H12 subsequently. 
trash = deck.load_labware('agilent_1_reservoir_290ml.json', 0)
samples = deck.load_labware('nest_96_wellplate_200ul_flat.json', slot = 2)
samples.manual_offset([[127.4, 115.4],[27.4, 115.4], [27.4, 178.4]], save = True)
stocks = deck.load_labware('corning_6_wellplate_16.8ml.json', slot = 3)
stocks.manual_offset([[256.8, 127.4], [178.8, 127.4], [178.8, 165.4]], save = True)

New manual offset applied to opentrons_96_tiprack_20ul
Manual offset applied, but not saved
New manual offset applied to nest_96_wellplate_200ul_flat
Manual offset applied, but not saved
New manual offset applied to corning_6_wellplate_16.8ml_flat
Manual offset applied, but not saved


In [48]:
print(type(samples))

<class 'science_jubilee.labware.Labware.Labware'>


In [4]:
deck.safe_z

64.69

In [2]:
axo = Machine(address = "192.168.1.4")

In [5]:
deck = axo.load_deck("lab_automation_deck")
tip_rack = axo.load_labware("opentrons_96_tiprack_20ul", 1)
tiprack.load_manualOffset()
P20 = Pipette.from_config(0, 'Pipette', 'P300_config.json')
trash = axo.load_labware('agilent_1_reservoir_290ml.json', 0)
#pipette = Pipette(0, "Pipette", brand = "opentron", model = "config", max_volume= 20, min_volume= 0, zero_position = (0,0,0),\
#    blowout_position=None, drop_tip_position = None, mm_to_ul = None)# model = "config")
samples = axo.load_labware('nest_96_wellplate_200ul_flat.json', 2)
samples.load_manualOffset()
stocks = axo.load_labware('corning_6_wellplate_16.8ml.json', 3)
stocks.load_manualOffset()
camera_well = axo.load_labware("cam_wellplate.json", slot = 2)
# m.load_tool(pipette) # configure the pipette for use on the m

New manual offset applied to opentrons_96_tiprack_20ul
New manual offset applied to nest_96_wellplate_200ul_flat
New manual offset applied to corning_6_wellplate_16.8ml_flat


In [74]:
type(stocks['B1'])

science_jubilee.labware.Labware.Well

In [35]:
axo.configured_tools

{0: 'Pipette', 1: 'Camera'}

In [6]:
axo.home_all()

In [8]:
# Load the camera tool
camera = Cam(1, "Camera", '192.168.1.2', 'Sun92023', 'axo', 'C:/science-jubilee/axo/data', 50)

axo.load_tool(camera)
axo.load_tool(P20)

In [9]:
axo.move_to(z=180)

In [15]:
deck._get_slots()

{'0': Slot(slot_index='0', offset=[14.9, 4.4], has_labware=False, labware=None),
 '1': Slot(slot_index='1', offset=[157.0, 5.8], has_labware=False, labware=None),
 '2': Slot(slot_index='2', offset=[14.3, 100.8], has_labware=False, labware=None),
 '3': Slot(slot_index='3', offset=[154.6, 100.1], has_labware=False, labware=None),
 '4': Slot(slot_index='4', offset=[17.1, 201.1], has_labware=False, labware=None),
 '5': Slot(slot_index='5', offset=[157.0, 201.9], has_labware=False, labware=None)}

In [18]:
axo.move_to(x=154.6, y =100.1)

In [10]:
axo.pickup_tool(camera)

In [10]:
camera_A1 = camera_well['A1']
#new_offset =  (0.0, 0.0, 0.0)
camera_well.offset
#Labware._getxyz(camera_A1)

[14.3, 100.8]

In [26]:
axo.move_to(x=150, y=150)

In [12]:
camera.capture_image(camera_well['A1'], save_dir='C:/science-jubilee/axo/data')

In [15]:
camera.capture_image(camera_well['H12'], save_dir='C:/science-jubilee/axo/data')

In [111]:
x_test, y_test, z_test = Labware._getxyz(camera_well['A1'])
print(x_test, y_test)
axo.move_to(x = x_test, y = y_test)

145.8 171.39999999999998


In [104]:
axo.move_to(x=14.3, y=108)

In [13]:
camera.take_picture()

Error: [Errno 2] No such file


In [16]:
axo.park_tool()